In [15]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import os
os.environ["OMP_NUM_THREADS"] = "1"


In [18]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [19]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [20]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [21]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_data = StandardScaler().fit_transform(df_market_data)

# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    scaled_data,
    columns=df_market_data.columns
)

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()



,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


In [22]:
# Plot the scaled data
df_market_data_scaled.hvplot.line(
    width=1100,
    height=400,
    rot=90,
    title='Cryptocurrency Market Data Scaled: Standarized Returns for Several Holding Periods (%)',
    ylabel= 'Standarized Return (%)',
    xlabel='Cryptocurrency Name',
    value_label='Standarized Return in the Period (%)'
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (Standarized Return in the Period (%))

---

### Find the Best Value for k Using the Original Data.

In [23]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k = range(1,12)
list(k)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [24]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. create a KMeans model using the loop counter for the n_clusters
# 2. fit the model to the scaled data
# 3. append the model.inertia_ to the inertia list


for i in k:
    model = KMeans(n_clusters=i)
    model.fit(df_market_data_scaled)
    inertia.append(model.inertia_)
    
inertia

C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak

[287.0,
 195.82021818036043,
 123.19048183836956,
 79.02243535120975,
 65.40592346140595,
 53.05778846567062,
 44.03296539684707,
 36.85147699909247,
 32.124834992493994,
 28.52090489705486,
 24.83751411826692]

In [25]:
# Create a dictionary with the data to plot the Elbow curve
dic_inertia = {'k':k, 'inertia':inertia}

# Create a DataFrame with the data to plot the Elbow curve
inertia_df = pd.DataFrame(dic_inertia)

inertia_df

,k,inertia
0,1,287.000000
1,2,195.820218
2,3,123.190482
3,4,79.022435
4,5,65.405923
5,6,53.057788
6,7,44.032965
7,8,36.851477
8,9,32.124835
9,10,28.520905


In [63]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
og_inertia_plot=inertia_df.hvplot(
    x      = 'k',
    y      = 'inertia',
    title  = 'Standarized Performance Data: Inertia versus Number of Clusters',
    xlabel = 'Number of Clusters (k)',
    ylabel = 'Inertia (units)'
)
og_inertia_plot

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** The best value for k can be determined visually by identifying the "elbow" point on which the plotted curve. This "elbow" point represents the point where the inertia starts decreasing at a slower rate. It indicates the optimal number of clusters. From the Elbow Curve above, the k value is shown as 4. 

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [64]:
# Initialize the K-Means model using the best value for k
kmodel = KMeans(n_clusters=4)

In [65]:
# Fit the K-Means model using the scaled data
kmodel.fit(df_market_data_scaled)

C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4)

In [67]:
# Predict the clusters to group the cryptocurrencies using the scaled data
clusters = kmodel.predict(df_market_data_scaled)

# View the resulting array of cluster values.
clusters

array([1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 3, 0, 0, 0, 0])

In [68]:
# Add a new column to the DataFrame with the predicted clusters
df_market_data_scaled['cluster_original'] = clusters

# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,cluster_original
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,1
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,1
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,1


In [69]:
# Add a new column to the DataFrame with the predicted clusters
df_market_data_scaled['cluster_original'] = clusters

# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,cluster_original
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,1
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,1
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,1


In [70]:
df_market_data_scaled.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,cluster_original
count,41.000000,4.100000e+01,4.100000e+01,4.100000e+01,4.100000e+01,4.100000e+01,4.100000e+01,41.000000
mean,0.000000,1.895503e-16,2.707861e-17,2.978647e-17,-5.415722e-18,-1.326852e-16,4.197185e-17,0.439024
std,1.012423,1.012423e+00,1.012423e+00,1.012423e+00,1.012423e+00,1.012423e+00,1.012423e+00,0.672636
min,-4.981042,-1.682027e+00,-2.217108e+00,-1.393153e+00,-9.560487e-01,-5.511464e-01,-2.963296e-01,0.000000
25%,-0.127467,-7.066688e-01,-6.299628e-01,-4.605580e-01,-5.517599e-01,-4.998478e-01,-2.817468e-01,0.000000
50%,0.077497,-1.906843e-01,-9.190922e-03,-6.103015e-02,-1.592496e-01,-3.550537e-01,-2.255326e-01,0.000000
75%,0.331280,4.931931e-01,6.435649e-01,1.165382e-01,1.606038e-02,-4.736110e-02,-1.454693e-01,1.000000
max,1.919812,2.572251e+00,2.907054e+00,5.351455e+00,4.769913e+00,4.632380e+00,6.088625e+00,3.000000


In [71]:

# Scatter plot using hvPlot
# Color points mark the clusters
# crypto name are in the hover
plot_original_clusters = df_market_data_scaled.hvplot.scatter(
    x     = "price_change_percentage_14d", 
    y     = "price_change_percentage_1y", 
    by    = 'cluster_original',
    title = "Cryptocurrencies Standarized Returns. K-Mean Clusters with k=4.",
    hover_cols = 'coin_id'
)
plot_original_clusters

:NdOverlay   [cluster_original]
   :Scatter   [price_change_percentage_14d]   (price_change_percentage_1y,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [72]:
# Create a PCA model instance and set `n_components=3`.
pca_model = PCA(n_components = 3)

# coming back to the original data
df_market_data_scaled_no_clusters = df_market_data_scaled.drop(columns='cluster_original')

In [73]:
# Use the PCA model with `fit_transform` to reduce to three principal components.
market_data_pca = pca.fit_transform(df_market_data_scaled_no_clusters)

# View the first five rows of the DataFrame. 
market_data_pca[0:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [76]:
# Retrieve the explained variance to determine how much information can be attributed to each principal component.
pca.explained_variance_ratio_

# Total explained variance with 3 components
pca.explained_variance_ratio_.sum()

0.8950316570309841

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 
The total explained variance of approximately 89.5% across the three principal components indicates a substantial retention of variability from the original dataset. This high percentage suggests that the reduced-dimensional representation effectively summarizes essential information from the original features, ensuring that the PCA transformation captures the underlying structure of the data while significantly reducing its dimensionality.

In [77]:
# Creating a new DataFrame with the PCA data
df_market_data_pca = pd.DataFrame(market_data_pca, columns=['PC1','PC2','PC3'])

# Copy the crypto names from the original data
df_market_data_pca['coin_id'] = df_market_data_scaled.index

# Set the coin_id column as index
df_market_data_pca = df_market_data_pca.set_index('coin_id')

# Display sample data
df_market_data_pca.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In [78]:
# Create a list with the number of k-values to try
# We use a range from 1 to 11
k = list(range(1,12))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [79]:
# Create an empy list to store the inertia values
inertia_pca = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model_pca = KMeans(n_clusters = i)
    model_pca.fit(df_market_data_pca)
    inertia_pca.append(model_pca.inertia_)

inertia_pca


C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak

[256.87408556789256,
 165.9019940203601,
 93.77462568057295,
 49.66549665179736,
 37.87874703346248,
 27.618971787957435,
 21.134056037473606,
 17.01696214007742,
 13.719376402257225,
 10.610616508357415,
 8.07035254543547]

In [80]:
# Create a dictionary with the data to plot the Elbow curve
inertia_pca = {'k':k, 'inertia_pca':inertia_pca}

# Create a DataFrame with the data to plot the Elbow curve
inertia_pca_df = pd.DataFrame(inertia_pca)

inertia_pca_df = inertia_pca_df.set_index('k')
inertia_pca_df

,inertia_pca
k,
1,256.874086
2,165.901994
3,93.774626
4,49.665497
5,37.878747
6,27.618972
7,21.134056
8,17.016962
9,13.719376


In [81]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
inertia_pca_plot = inertia_pca_df['inertia_pca'].hvplot(
    title  = 'Principal Components of the Standarized Performance: Inertia v/s Number of Clusters',
    xlabel = 'Number of Clusters (k)',
    ylabel = 'Inertia (units)'     
)

inertia_pca_plot

:Curve   [k]   (inertia_pca)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value for k, based on the first elbow point in the graph, is k = 4.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** The best value for k remains the same when using the PCA data as compared to the original data at k = 4. The analysis suggests that the underlying structure of the data is well preserved even after dimensionality reduction through PCA. Therefore, in this case, both analyses lead to the same optimal value for k, which indicates that k = 4 is the best choice for clustering the data.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [82]:
# Initialize the K-Means model using the best value for k
k = 4
model_pca = KMeans(n_clusters=k, random_state=1)

In [83]:
# Fit the K-Means model using the PCA data
model_pca.fit(df_market_data_pca)


C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4, random_state=1)

In [86]:
# Predict the clusters to group the cryptocurrencies using the PCA data
predicted_clusters = model_pca.predict(df_market_data_pca)

# View the resulting array of cluster values.
predicted_clusters[0:5]


array([2, 2, 0, 0, 2])

In [87]:
# Add a new column to the DataFrame with the predicted clusters
df_market_data_pca['cluster_pca'] = cluster

# Display sample data
df_market_data_pca.head()

,PC1,PC2,PC3,cluster_pca
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,2
ethereum,-0.458261,0.458466,0.952877,2
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,2


In [88]:
# Plotting the PCAs
pca_longer_term_plot=df_market_data_pca.hvplot.scatter(
    x  = 'PC1',
    y  = 'PC2',
    by = 'cluster_pca',
    hover_cols='coin_id',
    title  = "Principal Components K-Means Clusters and PC1 versus PC2",
    xlabel = 'First Principal Component \n Returns (%) (PC1)',
    ylabel = 'Second Principal Component \n Returns (%) (PC2)',
    width  = 600,
    height = 500
).opts(legend_position='top')

pca_longer_term_plot

:NdOverlay   [cluster_pca]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [89]:
# Composite plot to contrast the Elbow curves
og_inertia_plot + inertia_pca_plot

:Layout
   .Curve.I           :Curve   [k]   (inertia)
   .Curve.Inertia_pca :Curve   [k]   (inertia_pca)

In [90]:
# Composite plot to contrast the clusters
plot_original_clusters + pca_longer_term_plot

:Layout
   .NdOverlay.I  :NdOverlay   [cluster_original]
      :Scatter   [price_change_percentage_14d]   (price_change_percentage_1y,coin_id)
   .NdOverlay.II :NdOverlay   [cluster_pca]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:**  
Based on my analysis of the composite plot of the Elbow curves, which depicted the inertia values for both the original data and the PCA-transformed data across different values of k (number of clusters), several insights can be drawn regarding the impact of using fewer features to cluster the data using K-Means.

Firstly, when reducing the number of features using PCA before clustering, I noticed that the inertia values were generally higher compared to using the original feature set. This indicates that the clusters formed using the PCA-transformed data were less compact or less tightly grouped together than those formed using the original feature set. The higher inertia suggests that there is more dispersion within the clusters, possibly due to the loss of some information during the dimensionality reduction process.

Despite the higher inertia values for the PCA-transformed data, the optimal value of k remained consistent at 4 for both the original data and the PCA-transformed data. This suggests that, while reducing the dimensionality through PCA may lead to less distinct cluster assignments due to the loss of some information, the fundamental structure of the data, as reflected by the optimal number of clusters, remains relatively unchanged.

However, it's important to note that while the optimal value of k remained consistent, the interpretation of the clusters may vary between the original data and the PCA-transformed data. Clusters formed using the PCA-transformed data may be less tightly defined and may exhibit more overlap or ambiguity compared to clusters formed using the original feature set.

In conclusion, while reducing the dimensionality through PCA before clustering may result in clusters that are less compact and well-separated, the determination of the optimal number of clusters appears to be relatively robust. However, the trade-offs between dimensionality reduction and clustering performance need to be carefully evaluated based on the specific characteristics and requirements of the dataset, as well as the interpretability of the resulting clusters.